In [4]:
import os # operating system
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [5]:
#os.getcwd()
#os.chdir("path")

In [ ]:
# import the dataset
dataset = pd.read_excel('E-com_Data.xlsx')
dataset.head()

In [ ]:
dataset.shape

In [ ]:
# information of the dataset
dataset.info()

In [ ]:
# no. of unique
dataset.nunique()

In [ ]:
# Check missing data
dataset.isnull().sum()

In [ ]:
# Total number of percent
dataset.isnull().sum() / len(dataset)*100

In [ ]:
# we will drop missing data for customer (row wise)
# Data cleaning - duplicate data
dataset.loc[dataset.duplicated(),:]

In [ ]:
#dataset.head(20)

In [ ]:
# drop the duplicate data
dataset = dataset.drop_duplicates(ignore_index = True)

In [ ]:
dataset.shape

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset = dataset.dropna(subset=['CustomerID'])

In [ ]:
dataset.shape

In [ ]:
#dataset['Date'] = pd.to_datetime(dataset['Date of purchase'])

In [ ]:
#dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset = dataset.iloc[:,0:9]
dataset.head()

In [ ]:
dataset.head()

In [ ]:
dataset['Date'] = pd.to_datetime(dataset['Date of purchase'])
dataset['Date_1'] = dataset['Date']

In [ ]:
dataset.head(2)

In [ ]:
# Objective : Build an unsupervised learning model which can enable your 
# company to analyze their customers via RFM (Recency, Frequency and Monetary value) approach.

# Recency = Latest Date - Last invoice data
# latest date = today's date or 

# Frequency = # of invoice n. of the transaction done by customer

# Monetary = Sum of total amount purchased by customer

max(dataset['Date of purchase'])

In [ ]:
min(dataset['Date of purchase'])

In [ ]:
# last date = 2017-12-19
# latest data = 2017-12-20

import datetime as dt
Latest_Date = dt.datetime(2017,12,20)

In [ ]:
Latest_Date

In [ ]:
dataset.head()

In [ ]:
df_recency = dataset.groupby(['CustomerID'],as_index=False)['Date of purchase'].max()
df_recency.columns = ['CustomerID','Last_Purchase_Date']
df_recency['Recency'] = (Latest_Date - df_recency['Last_Purchase_Date']).dt.days
df_recency.drop(columns=['Last_Purchase_Date'],inplace=True)
FM_Table = dataset.groupby('CustomerID').agg({'InvoieNo'   : lambda x:len(x),'Price'  : lambda x:x.sum()})
FM_Table.rename(columns = {'InvoieNo' :'Frequency','Price':'Monetary_Value'},inplace= True)
RFM_Table = df_recency.merge(FM_Table,left_on='CustomerID',right_on='CustomerID')

In [ ]:
RFM_Table

In [ ]:
# Create RMF model by each customer

RMFScore = dataset.groupby('CustomerID').agg({'Date_1': lambda x : (Latest_Date - x.max()).days,
                                            'Date': lambda x : x.count(),
                                            'Price' : lambda x : x.sum()})

In [ ]:
RMFScore['Date'] = RMFScore['Date'].astype(int)
RMFScore.rename(columns = {'Date_1' : "Recency", "Date": "Frequency","Price":"Monetory"},
               inplace = True)

RMFScore.reset_index().head(100)

In [ ]:
# EDA Part : 
RMFScore.Recency.describe()

In [ ]:
# split the data into four segments using quantile method
quantiles = RMFScore.quantile(q=[0.25,0.50,0.75])
quantiles = quantiles.to_dict()

In [ ]:
quantiles

In [ ]:
# Create own function to define R, F and M Segment

def RScoring(x,p,d):
    if x<= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else :
        return 4
    
def FnMScoring(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75] :
        return 2
    else :
        return 1

In [ ]:
RMFScore.columns

In [ ]:
RMFScore['R'] = RMFScore['Recency'].apply(RScoring, args = ('Recency', quantiles,))
RMFScore['F'] = RMFScore['Frequency'].apply(FnMScoring, args = ('Frequency', quantiles,))
RMFScore['M'] = RMFScore['Monetory'].apply(FnMScoring, args = ('Monetory', quantiles,))

In [ ]:
RMFScore.head(20)

In [ ]:
RMFScore['RFMGroup'] = RMFScore.R.map(str) + RMFScore.F.map(str) + RMFScore.M.map(str)

In [ ]:
RMFScore.head(10)

In [ ]:
RMFScore['RFMScore'] = RMFScore[['R','F','M']].sum(axis=1)

In [ ]:
RMFScore.head(20)

In [ ]:
# Assign Loyality level to each customer

Loyalty_level = ['Diamond','Platinum','Gold','Silver']

score_cuts = pd.qcut(RMFScore.RFMScore, q = 4, labels = Loyalty_level)

RMFScore['RFM_Loyalty_level'] = score_cuts.values
RMFScore.reset_index().head(20)

In [ ]:
# validate the data for RFMGroup = 111

RMFScore[RMFScore['RFMGroup']=='111'].sort_values('Monetory', ascending=False).reset_index().head(25)

In [ ]:
RMFScore=RMFScore.reset_index()

In [ ]:
def handle_neg_n_zero(num):
    if num <=0 :
        return 1
    else :
        return num
#Apply handle_neg_n_zero function to Recency and Monetary columns 
RMFScore['Recency'] = [handle_neg_n_zero(x) for x in RMFScore.Recency]
RMFScore['Monetory'] = [handle_neg_n_zero(x) for x in RMFScore.Monetory]

#Perform Log transformation to bring data into normal or near normal distribution
Log_Tfd_Data = RMFScore[['Recency', 'Frequency', 'Monetory']].apply(np.log, axis = 1).round(3)

In [ ]:
RMFScore.columns

In [ ]:
# Recency and Frequency - not required as we don't have negative value
# RMFScore['Recency'] = [handle_neg_n_zero(x) for x in RMFScore.Recency]

In [ ]:
#RMFScore['Monetory'] = [handle_neg_n_zero(x) for x in RMFScore.Monetory]

In [ ]:
Log_Tfd_Data.head()

In [ ]:
newdata = RMFScore[['Recency', 'Frequency', 'Monetory']]

In [ ]:
newdata.head()

# Feature Scaling

In [ ]:
sns.boxplot(y = 'Monetory', data=RMFScore)

In [ ]:
from sklearn.preprocessing import StandardScaler
scalobj = StandardScaler()
scaled_data = scalobj.fit_transform(newdata)
scaled_data

In [ ]:
scaled_data = pd.DataFrame(scaled_data, index =RMFScore.index, columns= newdata.columns)

In [ ]:
scaled_data.head()

# Build Clustering Model 

In [ ]:
from sklearn.cluster import KMeans

sum_of_sq_dist = {}

for k in range(1,15):
    km = KMeans(n_clusters = k, init='k-means++', max_iter=300)
    km = km.fit(scaled_data)
    sum_of_sq_dist[k] = km.inertia_
    
# Visualisation for getting elbow method (to find the actual k value)

sns.pointplot(x = list(sum_of_sq_dist.keys()),
             y = list(sum_of_sq_dist.values()))
plt.xlabel("Number of Clusters(k)")
plt.ylabel("Sum of Square Distance(Euclidean Distance)")
plt.title("Elbow Method for Optimal K value")
plt.show()

In [ ]:
km = KMeans(n_clusters = 4, init='k-means++', max_iter=300)

In [ ]:
y_kmeans = km.fit_predict(scaled_data)

In [ ]:
y_kmeans

In [ ]:
RMFScore['Cluster'] = km.labels_
RMFScore.head()

In [ ]:
RMFScore.info()

In [ ]:
RMFScore.head(1)

In [ ]:
# Look at the centroids per cluster
centroid_df = RMFScore.groupby('RFM_Loyalty_level')[['Recency','Frequency','Monetory',
                                          'CustomerID']].agg({'Recency':'mean',
                                                             'Frequency':'mean',
                                                             'Monetory':'mean',
                                                             'CustomerID':'nunique'}).reset_index()

In [ ]:
# look at the centroids per cluster
centroid_df = RMFScore.groupby('Cluster')[['Recency', 'Frequency', 'Monetory','CustomerID']].agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetory':'mean','CustomerID':'nunique'}).reset_index()
centroid_df = centroid_df.rename(columns = {'CustomerID':'NumBuyers'})
centroid_df

In [ ]:
centroid_df['Percent'] = round(centroid_df['NumBuyers']/centroid_df.NumBuyers.sum()*100,2)
centroid_df

In [ ]:
centroid_df_1 = RMFScore.groupby('RFM_Loyalty_level')[['Recency', 'Frequency', 'Monetory','CustomerID']].agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetory':'sum','CustomerID':'nunique'}).reset_index()
centroid_df_1 = centroid_df_1.rename(columns = {'CustomerID':'NumBuyers'})
centroid_df_1

In [ ]:
centroid_df_1['Percent'] = round(centroid_df_1['NumBuyers']/centroid_df_1.NumBuyers.sum()*100,2)
centroid_df_1

In [ ]:
!pip install squarify
import squarify

In [ ]:
fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(20,15)
colors_dict ={'0': 'Purple','1':'royalblue','2':'red','3':'yellow'}

squarify.plot(sizes=centroid_df_1['NumBuyers'],
             color=colors_dict.values(),
             label=['{} \n{:.0f} customers \n({}%)'.format(*centroid_df_1.iloc[i])
             for i in range (0,len(centroid_df_1))], alpha=0.6)

plt.title("Customer Segments", fontsize=26, fontweight="bold")
plt.axis("off")

plt.show()

In [71]:
RMFScore.to_excel('Final_output.xlsx')